# Simple agent 

In [2]:
from dotenv import load_dotenv
from openai import OpenAI
from agents import Agent , Runner , trace ,function_tool
from openai.types.responses import ResponseTextDeltaEvent
import os
import asyncio

In [3]:
load_dotenv(override=True)

True

In [4]:
instruction1 = "You are Cricket Fanatic. You have extreme knowledge in cricket and includes records, statistics, player info and many other. Answer only to this context and nothing else .Kepp it concise"
instruction2 = "You are Bollywood Fanatic. You know all info about bollywood , celibrites, box office, release, actor and actress ifo etc etc. Answer only to this context and nothing else . Keep it Concise"


In [5]:
bollywood_agent=Agent(name="Bollywood crazy",model="gpt-4o-mini",instructions=instruction2)
cricket_agent=Agent(name="Cricket Agent",model="gpt-4o-mini",instructions=instruction1)

In [6]:
response=Runner.run(bollywood_agent,"Tell me no of films done by akshay kumar.Is he the highest paid?")
response
#its a co-routine


<coroutine object Runner.run at 0x114b25470>

In [7]:
result=await response # it executes coroutine
print(result)

RunResult:
- Last agent: Agent(name="Bollywood crazy", ...)
- Final output (str):
    As of now, Akshay Kumar has acted in over 150 films. He is one of the highest-paid actors in Bollywood, often ranking at the top of the list for his earnings per film and overall income.
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [8]:
from IPython.display import display,Markdown

display(Markdown(f" ### --BollywoodCrazy Channel -- \n\n #### {result.final_output} "))

 ### --BollywoodCrazy Channel -- 

 #### As of now, Akshay Kumar has acted in over 150 films. He is one of the highest-paid actors in Bollywood, often ranking at the top of the list for his earnings per film and overall income. 

In [9]:
response2= await Runner.run(cricket_agent,"virat kohli statistics in tabular format in T20I's, ODI AND TEST")
display(Markdown(f'''### Cricket Mania \n\n {response2.final_output} '''))

### Cricket Mania 

 Here's a summary of Virat Kohli's statistics across T20Is, ODIs, and Tests:

| Format  | Matches | Innings | Runs  | Average | Centuries | Fifties | Strike Rate |
|---------|---------|---------|-------|---------|-----------|---------|-------------|
| T20I    | 109     | 99      | 4008  | 52.73   | 1         | 37      | 138.22      |
| ODI     | 274     | 253     | 13348 | 57.75   | 47        | 96      | 87.50       |
| Test    | 111     | 194     | 8819  | 49.18   | 28        | 28      | 57.77       |

*Note: Statistics are subject to change as Kohli is an active player.* 

# Lets stream the output and add trace 

In [10]:
response2=Runner.run_streamed(cricket_agent,"Virat Kohli Statictics in Tabular format in T20, ODI, TEST")
async for event in response2.stream_events():
    print(event)
    

AgentUpdatedStreamEvent(new_agent=Agent(name='Cricket Agent', instructions='You are Cricket Fanatic. You have extreme knowledge in cricket and includes records, statistics, player info and many other. Answer only to this context and nothing else .Kepp it concise', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), type='agent_updated_stream_event')
RawResponsesStreamEvent(data=ResponseCreatedEvent(response=Response(id='resp_0e0d34631008d0eb0068e10d796fd481969e6e44f6fc347afe', cre

In [11]:
response2=Runner.run_streamed(bollywood_agent,"Bollywood history including first film ,bolywood talkise and more little descriptive")
async for event in response2.stream_events():
    if event.type=="raw_response_event" and isinstance(event.data,ResponseTextDeltaEvent):
        print(event.data.delta,end="",flush=True) #the end="" should be given else it takes \n and output will be very bad

Bollywood, the Hindi-language film industry based in Mumbai, has a rich history that dates back to the early 20th century. 

### First Film:
The first Indian feature film is "Raja Harishchandra," directed by Dadasaheb Phalke, released in 1913. This silent film marked the beginning of Indian cinema.

### The Talkies Era:
The transition to sound films, known as "talkies," began with "Alam Ara" in 1931. Directed by Ardeshir Irani, it was the first Indian film to have sound, revolutionizing the industry.

### Golden Era:
The 1950s to 1970s is often referred to as the Golden Era of Bollywood, characterized by the emergence of legendary filmmakers like Raj Kapoor and Satyajit Ray, and iconic actors such as Dilip Kumar, Rajesh Khanna, and Nargis.

### Modern Bollywood:
The 1990s introduced global audiences to Bollywood with films like "Dilwale Dulhania Le Jayenge." The industry expanded further with mega stars like Shah Rukh Khan, Salman Khan, and Aishwarya Rai becoming household names.

### 

In [12]:
import asyncio
message="Rahul dravid career statistics"
with trace("run parrallel LLM;S"):
    results=await asyncio.gather(
        Runner.run(bollywood_agent,message),
        Runner.run(cricket_agent,message)

    )

outputs=[result.final_output for result in results]
for output in outputs:
    print(output + "\n\n")

I'm focused on Bollywood and celebrities. If you have questions about films, actors, or anything related to the film industry, feel free to ask!


Rahul Dravid, often referred to as "The Wall," is one of India's greatest cricketers. Here are his career statistics:

### Test Career:
- **Matches**: 164
- **Innings**: 286
- **Runs**: 13,288
- **Average**: 52.31
- **Centuries (100s)**: 36
- **Half-Centuries (50s)**: 63
- **Highest Score**: 280

### One Day Internationals (ODIs):
- **Matches**: 344
- **Innings**: 318
- **Runs**: 10,889
- **Average**: 39.16
- **Centuries (100s)**: 12
- **Half-Centuries (50s)**: 39
- **Highest Score**: 153

### T20 Internationals:
- **Matches**: 1
- **Innings**: 1
- **Runs**: 0
- **Average**: 0.00
- **Centuries (100s)**: 0
- **Half-Centuries (50s)**: 0
- **Highest Score**: 0

### Notable Achievements:
- Inducted into the ICC Hall of Fame in 2018.
- Known for his solid technique and ability to play in difficult conditions.

Dravid is also renowned for his lead

In [13]:
AgentPicker=Agent(name="Response Picker",model='gpt-4o-mini',instructions="""You are best response picker. you analyse response returned by LLMS and pick the best one as per context and the topic of interest given 
     just pick the best response from llms in context of the topic and area of expertise being asked. If the llm response is outside the topic u donty pick that . You are intelligent and 
        and dont pick any random things. You see which llm had provided the topic answer and u chose that llm(agent) response. u know some llm may not provide the answer as it is outside their expertise. Choose the one which give the response for the 
            topic and output the response and reply with the best response. Please Prefix at beigining of response the following 
             channel_name:Channel1 if Bollywood related you picked or channel_name:Chhannel2 if cricket is picked  . 
             after this with a single new line response should foolow. 
             Lastly give the description why u picked it in one sentence   """
             )

In [14]:

import asyncio
message="Rahul dravid career statistics"
results=[]
with trace("run parrallel LLM;S"):
    results=await asyncio.gather(
        Runner.run(bollywood_agent,message),
        Runner.run(cricket_agent,message)

    )
#print(results)
response=[result.final_output for result in results]
#print(response)
topic_response=f" Response :{response} \n\n "
#print(topic_response)
best_response= await Runner.run(AgentPicker,topic_response)
print(f"Best rESPONSE :\n\n {best_response.final_output}")

Best rESPONSE :

 channel_name:Channel2

Rahul Dravid, one of cricket's legendary batsmen, has impressive career statistics:

### Test Career:
- **Matches:** 164
- **Innings:** 286
- **Runs:** 13,288
- **Batting Average:** 52.31
- **Centuries (100s):** 36
- **Half-centuries (50s):** 63
- **Highest Score:** 270

### One Day Internationals (ODIs):
- **Matches:** 344
- **Innings:** 318
- **Runs:** 10,889
- **Batting Average:** 39.16
- **Centuries (100s):** 12
- **Half-centuries (50s):** 83
- **Highest Score:** 153

### T20 Internationals:
- **Matches:** 1
- **Innings:** 1
- **Runs:** 0
- **Batting Average:** 0.00

Dravid is also renowned for his technique, temperament, and exceptional slip fielding. He served as the captain of India and is often referred to as "The Wall."

I picked this response because it provides extensive statistics about Rahul Dravid's cricket career, perfectly fitting the topic of cricket.


In [15]:
EntTool=bollywood_agent.as_tool(tool_name='Entertainment',tool_description="all information about bollywood")
SprtTool=cricket_agent.as_tool(tool_name="Sports",tool_description="Info about cricket sport")
SubChnlTool=AgentPicker.as_tool(tool_name="BestChannel",tool_description="Choosing best response and chhannel for topic")



In [16]:


@function_tool
def extract_tool_info(data:str):
    """Simple function to extract channel_name and response from tool call output"""
    if isinstance(data,dict) and 'text_output' in data:
        text_output=data['text_output']
    else:
        text_output=data
    
    lines = text_output.split('\n')
   
    channel = None
    response_lines = []
    
    for line in lines:
        # Extract channel name
        
        if 'channel_name' in line:
                channel = line.split(':')[1]

        # Extract Response
        elif line.strip() and not line.startswith('channel_name:'):
            response_lines.append(line.strip())
 
    
    response = '\n'.join(response_lines) if response_lines else ""

    if "Channel1" in channel:
        display(Markdown(f"## 🎬 **Bollywood Channel Selected**\n\n{response}"))
    elif "Channel2" in channel:
        display(Markdown(f"## 🏏 **Cricket Channel Selected**\n\n{response}"))
    
    return channel, response



In [17]:
tools=[EntTool,SprtTool,SubChnlTool,extract_tool_info]

In [18]:
tools

[FunctionTool(name='Entertainment', description='all information about bollywood', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Entertainment_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115811b20>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='Sports', description='Info about cricket sport', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Sports_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115811620>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='BestChannel', description='Choosing best response and chhannel for topic', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'stri

# Lets create a content Agent

In [28]:
instruction= """You are a content agent .Based on subject given to you , you leverage the below tools for the topic or subject 
            tools=EntertainmentTool ,SportsTool and extract_tool_info 
            DO NOT answer by yourself and DO NOT display by yourself. always use tool at your disposal to answer the question. You are just a guide      
            Follow the below instruction properly 
               1. Whatever question is asked by the user you need to pass to both the tools Entertainment Tool as well as Sports Tool 
               2. The response provided by the above tools should be passed to BestChannel tool and let the BestChannel tool decided the response to be picked \
               3. Finally use the tool extract_tool_info for genrating visually apealing display taking the input from BestChannel tool. ensure that channel is picked and exclude the description why this channel was picked
                critical rules:
                    1. Always use tool for performing tasks.
                    2. Use extract_tool_info for display. 
                    3. if there is an issue with any tool output the same and elaborate the issue for debugging"""
                       

In [29]:
content_agent=Agent(name='ContentAgent',model='gpt-4o-mini',tools=tools,instructions=instruction)

In [21]:
content_agent

Agent(name='ContentAgent', instructions='You are a content agent .Based on subject given to you , you leverage the below tools for the topic or subject \n            tools=EntertainmentTool ,SportsTool and extract_tool_info \n            DO NOT answer by yourself and DO NOT display by yourself. always use tool at your disposal to answer the question. You are just a guide      \n            Follow the below instruction properly \n               1. Whatever question is asked by the user you need to pass to both the tools Entertainment Tool as well as Sports Tool \n               2. The response provided by the above tools should be passed to BestChannel tool and let the BestChannel tool decided the response to be picked                3. Finally use the tool extract_tool_info for genrating visually apealing display \n                critical rules:\n                    1. Always use tool for performing tasks.\n                    2. Use extract_tool_info for display. \n                  

In [26]:
question="Akshay kumar career graph"
with trace("Content_Agent"):
    result=await Runner.run(content_agent,question)

## 🎬 **Bollywood Channel Selected**

Akshay Kumar, an iconic Bollywood actor, began his career in the late 1980s with small roles and gained significant recognition in the 1990s with action films like "Khiladi." Over the years, he diversified into various genres, including comedy and drama, starring in hits such as "Hera Pheri," "Dhadkan," and "Toilet: Ek Prem Katha." Kumar is known for his work ethic, often taking on several films each year, and has established himself as one of the highest-paid actors in India.
I picked this response because it provides a concise overview of Akshay Kumar's career trajectory, highlighting key milestones and his versatility as an actor.

trace :https://platform.openai.com/logs?api=traces

In [30]:
question="virat kohli all format statistics"
with trace("Content_Agent"):
    result=await Runner.run(content_agent,question)

## 🏏 **Cricket Channel Selected**

Virat Kohli is one of the leading cricketers in modern cricket. Here are his statistics across all formats:
**Test Matches:**
- Matches: 108
- Innings: 183
- Runs: 8,867
- Batting Average: 48.10
- 100s: 28
- 50s: 27
**One Day Internationals (ODIs):**
- Matches: 262
- Innings: 254
- Runs: 12,380
- Batting Average: 57.68
- 100s: 45
- 50s: 62
**T20 Internationals:**
- Matches: 115
- Innings: 99
- Runs: 3,759
- Batting Average: 52.74
- 100s: 1
- 50s: 34
These statistics reflect Kohli's proficiency and consistency across formats, making him one of the greatest batsmen in cricket history.
I picked this response because it directly answers the request for Virat Kohli's statistics in all formats of cricket, providing clear and relevant data.

### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across

In [46]:
instruction3="""you are an image generator . you recieve the response and channel_name from the tool extract_tool info. and u generate image 
of the personality or object related.You also add icon related to the topic  along with the display of response u also display the image beside it to give more appeal"""

In [91]:


@function_tool
def extractor_tool(data:str):
    """Simple function to extract channel_name and response from tool call output"""
    if isinstance(data,dict) and 'text_output' in data:
        text_output=data['text_output']
    else:
        text_output=data
    
    lines = text_output.split('\n')
   
    channel = None
    response_lines = []
    
    for line in lines:
        # Extract channel name
        
        if 'channel_name' in line:
                channel = line.split(':')[1]

        # Extract Response
        elif line.strip() and not line.startswith('channel_name:'):
            response_lines.append(line.strip())
 
    
    response = '\n'.join(response_lines) if response_lines else ""

 
    return channel, response



In [92]:
@function_tool
def content_display(args: str):
    ''' display contents visually apealing '''
    channel,response=args
    if "Channel1" in channel:
        display(Markdown(f"## 🎬 **Bollywood Channel Selected**\n\n{response}"))
    elif "Channel2" in channel:
        display(Markdown(f"## 🏏 **Cricket Channel Selected**\n\n{response}"))


In [94]:
tools=[content_display]

In [95]:
tools

[FunctionTool(name='content_display', description='display contents visually apealing', params_json_schema={'properties': {'args': {'title': 'Args', 'type': 'string'}}, 'required': ['args'], 'title': 'content_display_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x121111440>, strict_json_schema=True, is_enabled=True)]

In [ ]:



display_content_agent=Agent(name="display_content",
                            model="gemini-2.0-flash",
                            handoff_description="displaying agent",
                            tools=tools,
                            instructions=instruction3)

In [97]:
handoffs=[display_content_agent]

In [98]:
Tinstruction= """You are a content agent .Based on subject given to you , you leverage the below tools for the topic or subject 
            tools=EntertainmentTool ,SportsTool,BestChannel and extractor_tool
            DO NOT answer by yourself and DO NOT display by urself. always use tool at your disposal to answer the question. You are just a guide      
            Follow the below instruction properly 
               1. Whatever question is asked by the user you need to pass to both the tools Entertainment Tool as well as Sports Tool 
               2. The response provided by the above tools should be passed to BestChannel tool and let the BestChannel tool decided the response to be picked. The tool should also pick the channel name along with content
               3. Next use the tool extractor_tool for extracting channel_name and response taking the input from BestChannel tool.ensure that channel_name is picked 
               4. Finally Handoff for displaying: pass the response and channel_name to display_content_agent. the display_content_agent will take care of displaying visually appealing content
                critical rules:
                    1. Always use tool for performing tasks.
                    2. Use must handoff by passing the response and channel_name to display_content_agent 
                    3. if there is an issue with any tool or handoff ,output the same and elaborate the issue for debugging"""
                       

In [99]:
tools=[SprtTool,EntTool,SubChnlTool,extractor_tool]


In [100]:
tools

[FunctionTool(name='Sports', description='Info about cricket sport', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Sports_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115811620>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='Entertainment', description='all information about bollywood', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Entertainment_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115811b20>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='BestChannel', description='Choosing best response and chhannel for topic', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'stri

In [101]:
content_manager=Agent(
                        name="Content Manager",
                        model="gpt-4o-mini",
                        instructions=Tinstruction,
                        handoffs=handoffs,
                        tools=tools
)

In [102]:
message="info about virat kohli"
with trace("ContentDataManager"):
    result=await Runner.run(content_manager,message)

Error getting response: Error code: 400 - {'error': {'message': "The requested model 'gemini-2.0-flash' does not exist.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}. (request_id: req_0c82c33c55bd9e749c9b5abba359c3e8)


BadRequestError: Error code: 400 - {'error': {'message': "The requested model 'gemini-2.0-flash' does not exist.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}

check 2_lab2.ipynb
